# Time Series Prediction with Prophet 

## By: Jeff Hale

## Plan

### Explore the Prophet time series prediction library with several data sets to see how it performs with different parameters.

## Background
SARIMA and exponential smoothing (Holt Winters) are proven to work well for a range of time series prediction problems, but they require a good bit of parameter tuning. 

Prophet is a relatively new library -  it was released in 2017. It's designed for "Business Time Series".  Prophet is open source and was developed by Facebook. Facebook claims to use it a good deal internally. It is useful for univariate prediction. Prophet has APIs for Python and R. It is designed to be robust to use and to handle seasonal trends and holidays well. 

Prophet uses a generalized additive model, a type of regression model, to make predictions. It can accomodate non-linear smoothers applied ot the regressors. The model is decomposable into trend, seasonality, and holiday components. Prophet is curve-fitting instead of a model class like ARIMA that explicitly accounts for the temporal component of the model through autoregression. 

Prophet can accomodate expert information, so it's Bayesian-friendly.

Prophet is fast and the [Prohpet docs](https://facebook.github.io/prophet/docs/quick_start.html) are quite nice. Here's an [introductory paper](https://peerj.com/preprints/3190/) on the library. 

I've only found two evaluations of Prophet. One, discussed [here],(http://kourentzes.com/forecasting/2017/07/29/benchmarking-facebooks-prophet/) found it didn't perform fabulously with a relatively small amout of data, which isn't shocking. A [second analysis](https://pythondata.com/stock-market-forecasting-with-prophet/) of stock data also didn't find it to perform super well.

We're going to test it on smaller and larger data sets for business-case type problems.

## Set-up
Load the necessary libraries.
Configure the Jupyter Notebook settings.
Load the data into a pandas DataFrame

In [ ]:
# essentials
import numpy as np 
import pandas as pd 

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# time series algorithm
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics

# reproducibility
np.random.seed(34)

# Jupyter magic
%reload_ext autoreload
%autoreload 2
%matplotlib inline

sns.set()

In [ ]:
!ls              # list the file in the working directory

# Shampoo Sales

The first dataset is for shampoo sales. Available at [DataMarket](https://datamarket.com/data/set/22r0/sales-of-shampoo-over-a-three-year-period#!ds=22r0&display=line), original dataset Makridakis, Wheelwright, and Hyndman (1998). Hyndman, R.J. “Time Series Data Library”, https://datamarket.com/data/list/?q=provider:tsdl. Accessed on 12/12/18.

This dataset contains monthly data over 3 years for shampoo sales. So 36 observations total.

I found these datasets through Jason Brownlee's [Machine Learning Mastery](https://machinelearningmastery.com).

In [ ]:
!pip list      # list the package version numbers for reproducibiity

We need to skip the header row and exclude the text at the bottom of the .csv file. 

Prophet requires the date column to be labeled *ds* and the target column to be labeled *y*. 

The date column isn't a standard format Pandas will be able to convert, so we'll need to make a date custom parser. We'll do that in a bit.

In [ ]:
df_shampoo_orig = pd.read_csv('../input/sales-of-shampoo-over-a-three-year-period/sales-of-shampoo-over-a-three-ye.csv', 
                              nrows=36,
                              skiprows = 1, 
                              names = ['ds', 'y'], 
                              parse_dates = True )
df = df_shampoo_orig
df

In [ ]:
df.info()

Let's get the *ds* column into datatime format.

In [ ]:
df['ds'] = df.ds.apply(lambda x: "198"+x)
df.ds.head()

### tseries.offset.MonthEnd(0)
This next cell uses the awesome tseries.offset.MonthEnd(0) method to make the day of the month the final day. This is necessary for Prophet to make the predictions we need at the correctly spaced monthly intervals.

In [ ]:
df['ds']=pd.to_datetime(df['ds'])+pd.tseries.offsets.MonthEnd(0)

In [ ]:
df.head()

That looks better.

## Prophet forecast

We need to make a dataframe of future dates with the *.make_future_dataframe* method. Let's make 12 months worth of predictions.

In [ ]:
train = df[:24]
train.tail()

Let's instantiate a Prophet object and fit it to the training data.

Prophet tries to model daily and weekly seasonality by default. We'll define our own yearly seasonal pattern. 

"Seasonalities are estimated using a partial Fourier sum... a partial Fourier sum can approximate an aribtrary periodic signal. The number of terms in the partial sum (the order) is a parameter that determines how quickly the seasonality can change." - from the [docs](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html).

We'll try a Fourier sum of 5 to start and then adjust to see the effects.

We need to pass a n_changepoints parameter because the datset has < 25 observations, as discussedi n [this GitHub issue](https://github.com/facebook/prophet/issues/248#issuecomment-314624770).

In [ ]:
m = Prophet(weekly_seasonality=False, daily_seasonality=False, n_changepoints=2)
m.add_seasonality(name='yearly', period=12, fourier_order=5)
m.fit(train)

Let's make the future data frame and make the predictions. We need to pass *freq='m'* because we want monthly predictions.

In [ ]:
future = m.make_future_dataframe(periods=12, freq='M')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

Note that Prophet's make_future_dataframe method makes a DataFrame that includes the historical data, as well as the predictions. 

The predictions look reasonable at first glance. Let's make at a plot.

In [ ]:
figure = m.plot(forecast)

The dots are the actual data points. The line through the dots is the predicted values. The shared area represents the uncertainty intervals. 

Let's decompose this graph into the trend and seasonality.

In [ ]:
fig_decompose = m.plot_components(forecast)

The trend line looks correct. But the seasonality looks like it might be over-fitting.

### Reduce fourier_order
Let's see what happens if we reduce the *fourier_order* to 1.

In [ ]:
m2 = Prophet(weekly_seasonality=False, daily_seasonality=False, n_changepoints=2)
m2.add_seasonality(name='yearly', period=12, fourier_order=1)

m2.fit(train)
future2 = m2.make_future_dataframe(periods=12, freq='m')
forecast2 = m2.predict(future2)
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig2 = m2.plot(forecast2)

In [ ]:
fig2_decompose = m2.plot_components(forecast2)

Looks fairly similar, with a bit less over-fitting to the seasonality.

## Performance metrics

Let's compare the predictions to the actual values from the first forecast, when the fourier_term was higher and nothing was log transformed. 

We first need to create a cutoff column so Prophet knows when to compare predicted values to actual values.

In [ ]:
forecast['cutoff'] = pd.to_datetime('1980-12-31')
forecast['y'] = df['y']
forecast.tail()

In [ ]:
df_p = performance_metrics(forecast)
df_p.head()

Ok. Prophet is giving us a row of error terms for a variety of time windows into the future.  Let's see how these error values compare to the predictions with the lower Fourier term.

In [ ]:
forecast2['cutoff'] = pd.to_datetime('1980-12-31')
forecast2['y'] = df['y']
forecast2.tail()

In [ ]:
df_p2 = performance_metrics(forecast2)
df_p2.head()

Let's make a DataFrame that subtracts the error terms from the first predictions from the second predictions so that we can see which version predicted better. We'll make the horizons into indexes for legibility.

In [ ]:
df_p.index = df_p['horizon']
df_p2.index = df_p2['horizon']

df_error_compare = df_p - df_p2
df_error_compare = df_error_compare.drop(columns=['horizon', 'coverage'])
df_error_compare.loc[:'365 days']

Mostly we see negative numbers, which means that the second model with the Fourier term equal to 1 had larger error terms. So, in this case, the larger Fourier term model performed better.

Let's see how our second Prophet model forecast compared to a persistence forecast. A persistence forecast means a prediction that the final sales term in the training data would continue each month going forward. We'll make a DataFrame with a persistence forecast prediction.

In [ ]:
forecast_persist = forecast2.copy()
forecast_persist['cutoff'] = pd.to_datetime('1980-12-31')
forecast_persist['y'] = df['y']
forecast_persist['yhat'] = df.at[23,'y']
forecast_persist.tail()

Now let's compare the persistence forecast to the actual results.

In [ ]:
df_persist = performance_metrics(forecast_persist)
df_persist.head()

Now let's compute the difference between the persistence forecast error terms and the second Prophet model.

In [ ]:
df_persist.index = df_persist['horizon']

df_error_compare_persist = df_persist - df_p2
df_error_compare_persist = df_error_compare_persist.drop(columns=['horizon', 'coverage'])
df_error_compare_persist.loc[:'365 days']

Those numbers are all positive, meaning that the error terms were larger for the persistence model than for the Prophet model! Granted, this is a very small sample with a clear trend, but Prophet beat the baseline. That's good. Let's try another prediction problem.

# Airline Passenger Counts

The second dataset is for International airline passengers: monthly totals in thousands. Jan 49 – Dec 60 Available at [DataMarket](http://datamarket.com/data/list/?q=provider:tsdl), original dataset Source: Box & Jenkins (1976). Accessed on 12/16/18.

In [ ]:
df_air_orig = pd.read_csv('../input/internationalairlinepassengers/international-airline-passengers.csv', 
                              nrows=144,
                              skiprows = 1, 
                              names = ['ds', 'y'], 
                              parse_dates = True )
df_air = df_air_orig
df_air.head()

In [ ]:
df_air.tail()

In [ ]:
df_air.info()

No nulls. That's good. Looks like we need to parse the dates again.

In [ ]:
df_air['ds']=pd.to_datetime(df_air['ds'])+pd.tseries.offsets.MonthEnd(0)

In [ ]:
df_air.head()

In [ ]:
df_air.tail()

That all looks correct.

In [ ]:
df_air.info()

Ok. Now we're flying. Ha! Groan. We need to decide what time period we want to forecast. Let's forecast the last two years. 

Let's fit a Prophet model with the training data.

In [ ]:
train = df_air[:120]

Let's instantiate a Prophet object and fit it to the training data. Unlike the shampoo sales data, this time we're going to use the default Prophet parameters.

In [ ]:
m = Prophet()
m.fit(train)

Let's make the future data frame and make the predictions. We need to pass *freq='m'* because we want monthly predictions.

In [ ]:
future = m.make_future_dataframe(periods=24, freq='m')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

That data looks reasonable at first glance. Let's make at a plot.

In [ ]:
fig = m.plot(forecast)

Again, the dots are the actual data points. The line through the dots is the predicted values. The shared area represents the uncertainty intervals. 

Let's decompose this graph into the trend and seasonality. Before making any adjustments to the model.

In [ ]:
fig_decompose = m.plot_components(forecast)

In [ ]:
## Multiplicative seasonality

It looks like the latter seasonal high and low data points in the training data weren't picked up super well. Perhpas we should try making that seasonality effect multiplicative. 

So this is funny, I went to the docs to see how to add multiplicative seasonality and found a chart much like the one above. Turns out the Prophet team used this dataset as an example for multiplicative seasonality! Guess my instinct was right :) 

In [ ]:
m2 = Prophet(seasonality_mode='multiplicative')
m2.fit(train)

In [ ]:
future2 = m2.make_future_dataframe(periods=24, freq='m')
forecast = m2.predict(future2)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

In [ ]:
fig = m.plot(forecast)

That looks much better!

In [ ]:
fig = m.plot_components(forecast)

Looks like people travel in the summer.

## Model evaluation

Let's look at how the model with the multiplicative seasonality effect performs compared to a persistence model.

First we'll make a persistence model DataFrame.

In [ ]:
forecast_persist = forecast.copy()
forecast_persist['cutoff'] = pd.to_datetime('1959-01')
forecast_persist['y'] = df_air['y']
forecast_persist['yhat'] = df_air.at[119,'y']
forecast_persist.tail()

Here's what the persistence forecast looks like graphically.

In [ ]:
fig = m.plot(forecast_persist)

The straight line is the persistence forecast based on the value at the last training month, December 1958. The predicted range of values for the Prophet model is in the shaded region.

Ok. Now we need to add the *cutoff* and *y* values to the forecast.

In [ ]:
forecast['cutoff'] = pd.to_datetime('1958-12-31')
forecast['y'] = df_air['y']
forecast.tail()

Looks good. Now we need to compute the error terms for both models.

In [ ]:
df_air_p = performance_metrics(forecast[120:])
df_air_p.head()

In [ ]:
df_persist_p = performance_metrics(forecast_persist[120:])
df_persist_p.head()

Let's plot the RMSE lines for the two models. I chose RMSE because its a common metric that outputs an error term in the same units as the predicted variable. So in this case, passengers.

In [ ]:
df_air_plot = pd.DataFrame([df_air_p['rmse'], df_persist_p['rmse']])
df_air_plot = df_air_plot.T
df_air_plot.columns = ['prophet_rmse', 'persist_rmse']
df_air_plot.head()

In [ ]:
df_plot = df_air_plot[:12]
df_plot

In [ ]:
ax = sns.lineplot(
    data=df_plot,
    x=list(range(12)), 
    y='prophet_rmse',
    )
plt.title('RMSE Comparison of Prophet Model for Flight Passenger')
plt.xlabel('Month')
plt.ylabel('RMSE')

Let's add the persistence line and clean things up a bit.

In [ ]:
ax = sns.lineplot(
    data=df_plot,
    x=list(range(1, 13)), 
    y='prophet_rmse',
    )

ax = sns.lineplot(
    data=df_plot,
    x=list(range(1, 13)), 
    y='persist_rmse',
    )

plt.title('RMSE Comparison of Prophet Model for Flight Passenger')
plt.xlabel('Month')
plt.ylabel('RMSE')

plt.rcParams['figure.figsize']=(12, 6)
plt.legend(['Prophet RMSE','Persistence RMSE'])

The Persistence model has a higher RMSE for every time period. So Prophet beats the Persistence model! That's a low bar, but we'll take it - predicting the future isn't easy.

Now we will subtract one DataFrame from the other to see the exact differences in errors.

In [ ]:
df_air_compare = df_persist_p - df_air_p

df_air_compare = df_air_compare.drop(columns=['horizon', 'coverage'])

In [ ]:
df_air_compare[:12]

Confirming what we saw in our graph, we see that the persistence model had a larger error than the Prophet model for nearly all time periods and nearly all error terms.

## Future directions

It would be interesting to look at Propher performance on more and larger data sets. It would also be cool to compare Prophet with SARIMA and exponential smoothing (Holt-Winters) models. Deep learning models haven't proven especially effective at time series forecasting, but they could be compared also.

Overall, Prophet is fun to work with and Facebook claims to find it quite effective, so it merits further study. 

## If you found this helpful, please upvote this Kaggle Kernel so others can find it too.